In [1]:
import pandas as pd
import numpy as np
from stargazer.stargazer import Stargazer
import statsmodels.api as sm

In [2]:
infr = pd.read_csv('infrast_data.csv', sep=';')
infr.what.unique()

array(['Traffic_light', '4-ways intersection', 'Curve', 'Road_immission',
       'Turn_left', 'Turn_right'], dtype=object)

In [3]:
infr

,what,lat,lon
0,Traffic_light,53.551073,9.970167
1,Traffic_light,53.550523,9.969615
2,Traffic_light,53.550082,9.969438
3,Traffic_light,53.546138,9.969437
4,Traffic_light,53.545436,9.972425
...,...,...,...
72,Turn_right,53.552747,9.985368
73,Turn_right,53.557635,9.983789
74,Turn_right,53.557595,9.993096
75,Turn_right,53.557573,9.976766


In [4]:
dist_v = pd.read_csv('distance_platoon.csv')
dist_v.head()

,PS_JC,JC_RW,RW_DW,DW_AA,AA_AK
0,23.255813,10.970501,3.916299,4.820478,5.776278
1,24.907458,10.970501,3.916299,4.820478,5.776278
2,28.401357,10.970501,3.916299,4.820478,5.776278
3,32.055915,10.970501,3.916299,4.820478,5.776278
4,37.266453,10.970501,3.916299,4.820478,5.776278


In [5]:
dist_coord = pd.read_csv('platoon_distances_coord.csv')
dist_coord.head()

,v1_lat,v1_lon,v2_lat,v2_lon,v3_lat,v3_lon,v4_lat,v4_lon,v5_lat,v5_lon
0,53.555853,9.972764,53.555833,9.972746,53.555815,9.972724,53.555801,9.972685,53.555775,9.972733
1,53.555853,9.972764,53.555833,9.972746,53.555815,9.972724,53.555801,9.972685,53.555775,9.972733
2,53.555858,9.972760,53.555842,9.972725,53.555815,9.972724,53.555801,9.972685,53.555775,9.972733
3,53.555858,9.972760,53.555842,9.972725,53.555815,9.972724,53.555801,9.972685,53.555775,9.972733
4,53.555853,9.972764,53.555833,9.972746,53.555815,9.972724,53.555801,9.972685,53.555775,9.972733


In [6]:
v1 = pd.read_csv('v1.csv')
v2 = pd.read_csv('v2.csv')
v3 = pd.read_csv('v3.csv')
v4 = pd.read_csv('v4.csv')
v5 = pd.read_csv('v5.csv')

In [7]:
v1 = v1.drop('Unnamed: 0',1)
v2 = v2.drop('Unnamed: 0',1)
v3 = v3.drop('Unnamed: 0',1)
v4 = v4.drop('Unnamed: 0',1)
v5 = v5.drop('Unnamed: 0',1)

C:\Users\marco\AppData\Local\Temp\ipykernel_16724\1394089028.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  v1 = v1.drop('Unnamed: 0',1)
C:\Users\marco\AppData\Local\Temp\ipykernel_16724\1394089028.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  v2 = v2.drop('Unnamed: 0',1)
C:\Users\marco\AppData\Local\Temp\ipykernel_16724\1394089028.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  v3 = v3.drop('Unnamed: 0',1)
C:\Users\marco\AppData\Local\Temp\ipykernel_16724\1394089028.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  v4 = v4.drop('Unnamed: 0',1)
C:\Users\marco\AppData\Local\Temp\ipykernel_16724\1394089028.py:5: FutureWarning

In [8]:
v1.columns = infr.what
v2.columns = infr.what
v3.columns = infr.what
v4.columns = infr.what
v5.columns = infr.what

In [9]:
vlist = []
vlist.append(v1)
vlist.append(v2)
vlist.append(v3)
vlist.append(v4)
vlist.append(v5)

In [10]:
all_X = []
all_y = []

for k in range(len(vlist)):
    feats = []
    v = vlist[k]
    infrast = v.idxmin(axis=1)
    for i in range(len(v)):
        if (v.iloc[i][str(infrast[i])] < 80).any():
            feats.append([infrast[i]])
        else:
            feats.append([])
    new_v = pd.get_dummies(pd.DataFrame(feats), prefix='Feat')

    X = new_v.copy()
    y = dist_v[str(dist_v.columns[k])]
    X['old_dist'] = dist_v[str(dist_v.columns[k])].shift(10).fillna(0)
    all_X.append(X)
    all_y.append(y)

all_X_ = pd.concat(all_X).reset_index(drop=True).fillna(0)
all_y_ = pd.concat(all_y).reset_index(drop=True).fillna(0)
model_dist = sm.OLS(y, X).fit()
#predictions = model.predict(X)
model_dist.summary()

results_summary = model_dist.summary()
df = pd.DataFrame.from_records(results_summary.tables[1].data)
header = df.iloc[0]
df = df[1:] # take the data less the header row
df.columns = header
df.to_csv(f'../../results/Dist/platoon_6.csv')

In [11]:
platoon_all_CO2_ = pd.read_csv('platoon_CO2.csv')

In [12]:
platoon_all_CO2_.head(2)

,PS_JC,JC_RW,RW_DW,DW_AA,AA_AK
0,0.0,0.004662,0.004662,0.001198,0.00233
1,0.0,0.004662,0.004662,0.001198,0.00233


In [13]:
all_X = []
all_y = []

for k in range(len(vlist)):
    feats = []
    v = vlist[k]
    infrast = v.idxmin(axis=1)
    for i in range(len(v)):
        if (v.iloc[i][str(infrast[i])] < 100).any():
            feats.append([infrast[i]])
        else:
            feats.append([])
    new_v = pd.get_dummies(pd.DataFrame(feats), prefix='Feat')

    X = new_v.copy()
    y = platoon_all_CO2_[str(platoon_all_CO2_.columns[k])]
    X['old_dist'] = dist_v[str(dist_v.columns[k])].shift(10).fillna(0)
    all_X.append(X)
    all_y.append(y)

all_X_ = pd.concat(all_X).reset_index(drop=True).fillna(0)
all_y_ = pd.concat(all_y).reset_index(drop=True).fillna(0)
model_co2 = sm.OLS(y, X).fit()
#predictions = model.predict(X)
model_co2.summary()

results_summary = model_co2.summary()
df = pd.DataFrame.from_records(results_summary.tables[1].data)
header = df.iloc[0]
df = df[1:] # take the data less the header row
df.columns = header
df.to_csv(f'../../results/CO2/platoon_6.csv')

In [14]:
stargazer = Stargazer([model_dist,model_co2])
#stargazer.custom_columns(ts,[1]*len(ts))
stargazer.show_model_numbers(False)
stargazer.show_degrees_of_freedom(False)
with open('../../latex/1.tex','w') as f:
    f.write(stargazer.render_latex())
stargazer

In [16]:
# regression on individual couples
for k in range(len(vlist)):
    feats = []
    v = vlist[k]
    infrast = v.idxmin(axis=1)
    for i in range(len(v)):
        if (v.iloc[i][str(infrast[i])] < 80).any():
            feats.append([infrast[i]])
        else:
            feats.append([])
    new_v = pd.get_dummies(pd.DataFrame(feats), prefix='Feat')

    X = new_v.copy()
    yd = dist_v[str(dist_v.columns[k])]
    yco2 = platoon_all_CO2_[str(platoon_all_CO2_.columns[k])]
    X['old_dist'] = dist_v[str(dist_v.columns[k])].shift(10).fillna(0)
    
    model_dist = sm.OLS(yd, X).fit()
    model_co2 = sm.OLS(yco2, X).fit()
    
    df_dist = pd.DataFrame.from_records(model_dist.summary().tables[1].data)
    header = df_dist.iloc[0]
    df_dist = df_dist[1:] # take the data less the header row
    df_dist.columns = header
    df_dist.to_csv(f'../../results/Dist/platoon_6_{k}.csv')
    
    df_co2 = pd.DataFrame.from_records(model_co2.summary().tables[1].data)
    header = df_co2.iloc[0]
    df_co2 = df_co2[1:] # take the data less the header row
    df_co2.columns = header
    df_co2.to_csv(f'../../results/CO2/platoon_6_{k}.csv')
    